In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm
import seaborn as sns
import pickle
from tqdm import tqdm

from sklearn.preprocessing import normalize

sns.set()
sns.set_style('whitegrid')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
ntrain = 10000
ntest = 2000
d = 200
wa, wb = 2, 1
sigma = 2.0
lamb = 1.0
# lamb=0.0

ndropout = 20000

In [ ]:
# np.random.seed(42)
w = wa*np.random.rand(d, 1)-wb

In [ ]:
def generate_data(w, sigma, ntrain, ntest):
    n = ntrain + ntest
    X = np.random.randn(n, len(w))
    # X = normalize(X, axis=1, norm='l2')
    y = X@w + np.random.randn(n, 1)*sigma
    X_train, X_test = X[:ntrain, :], X[ntrain:, :]
    y_train, y_test = y[:ntrain], y[ntrain:]
    return X_train, X_test, y_train, y_test

def lr_sol(X, y, lamb):
    return np.linalg.inv(X.transpose()@X + lamb*np.eye(X.shape[1]))@X.transpose()@y

def mse_gap(X, y, p, lamb):
    # diag = np.sqrt(np.diag(X.transpose()@X)/X.shape[0])
    # diag = np.diag(X.transpose()@X)/X.shape[0]
    diag = np.diag(np.diag(X.transpose()@X))
    w = lr_sol(X, y, lamb)
    diag = w.transpose()@diag@w/X.shape[0]
    return (2*p*(1-p)*diag/(1+lamb)).reshape(-1)
    # return 2*p*(1-p)*((diag@w)**2).mean()/(1+lamb)
    # return 2*p*(1-p)*((X.transpose()@y)**2).mean()/(1+lamb)
    # return 2*p*(1-p)*(w**2).mean()/(1+lamb)

In [ ]:
X_train, X_test, y_train, y_test = generate_data(w, sigma, ntrain, ntest)

In [ ]:
what = lr_sol(X_train, y_train, lamb)

In [ ]:
print('estimation error on w = {:.4f}'.format(np.mean((w-what)**2)))

In [ ]:
loss_lr_train, loss_lr_test = np.mean((X_train@what-y_train)**2), np.mean((X_test@what-y_test)**2)

In [ ]:
print('Train MSE = {:.4f}, Test MSE = {:.4f}'.format(loss_lr_train, loss_lr_test))

In [ ]:
drprate = np.linspace(0.0, 0.5, 11)
loss_drp = np.zeros((len(drprate), ndropout))
loss_mse_drp = np.zeros((len(drprate), ))

for i, drp in tqdm(enumerate(drprate)):
    drp_msk = np.random.binomial(size=(d, ndropout), n=1, p=1-drp)
    wdrp = what*drp_msk
    loss_drp[i, :] = np.mean((X_test@wdrp - y_test)**2, axis=0)
    loss_mse_drp[i] = (((1-drp)*X_test@what - y_test)**2).mean()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 3))

ax.errorbar(x=np.arange(len(drprate)), y=loss_drp.mean(axis=1)-loss_mse_drp, yerr=loss_drp.std(axis=1), 
            fmt='s', capsize=3, capthick=3, label='Monte Carlo')
ax.plot(np.arange(len(drprate)), mse_gap(X_test, y_test, drprate, lamb), label='Theory', 
        marker='o', linestyle='', markersize=10)

ax.set_xticks(np.arange(len(drprate)));
ax.set_xticklabels(['{:.2f}'.format(p) for p in drprate]);
ax.set_xlabel('Dropout Rate')
ax.set_ylabel('MSE');
ax.legend(loc='best');

#### Iterate number of dimensions

In [ ]:
ntrain = 10000
ntest = 2000
# dim = np.arange(10, 210, 10)
dim = np.array([10, 50, 100, 200])
drprate = np.linspace(0.0, 0.5, 11)
wa, wb = 2, 1
sigma = 2.0
lamb = 1.0
# lamb=0.0

ndropout = 20000

In [ ]:
loss_drp = np.zeros((len(dim), len(drprate), 2)) # mean and var
loss_mse_drp = np.zeros((len(dim), len(drprate), ))
loss_theory = np.zeros((len(dim), len(drprate), ))

for i, d in tqdm(enumerate(dim)):
    w = wa*np.random.rand(d, 1)-wb
    X_train, X_test, y_train, y_test = generate_data(w, sigma, ntrain, ntest)
    what = lr_sol(X_train, y_train, lamb)
    
    
    # loss_drp = np.zeros((len(drprate), ndropout))
    # loss_mse_drp = np.zeros((len(drprate), ))

    for j, drp in enumerate(drprate):
        drp_msk = np.random.binomial(size=(d, ndropout), n=1, p=1-drp)
        wdrp = what*drp_msk
        
        loss_temp = np.mean((X_test@wdrp - y_test)**2, axis=0)
        
        loss_drp[i, j, 0] = loss_temp.mean()
        loss_drp[i, j, 1] = loss_temp.std()
        
        loss_mse_drp[i, j] = (((1-drp)*X_test@what - y_test)**2).mean()
        loss_theory[i, j] = mse_gap(X_test, y_test, drp, lamb)
        
        # loss_temp = np.mean((X_test@wdrp - y_test)**2, axis=0)
        # loss_drp[i, :] = np.mean((X_test@wdrp - y_test)**2, axis=0)
        # loss_mse_drp[i] = (((1-drp)*X_test@what - y_test)**2).mean()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 4))
# colors = [['orange', 'orangered'], ['forestgreen', 'darkgreen'], ['blue', 'navy'], ['darkviolet', 'indigo']]
colors = ['lightsteelblue', 'royalblue', 'mediumblue', 'midnightblue']

for i in range(len(dim)):
    ax.errorbar(x=np.arange(len(drprate)), y=loss_drp[i, :, 0]-loss_mse_drp[i, :], yerr=loss_drp[i, :, 1], 
                fmt='s', capsize=3, capthick=3, color=colors[i])
    ax.scatter(np.arange(len(drprate)), loss_theory[i, :], marker='s', facecolors='none', 
               edgecolors=colors[i], s=100)
    
## for marker
ax.scatter(-10, -10, marker='s', facecolors='none', edgecolors='dimgray', s=100, label='Theory')
ax.errorbar(x=-10, y=-10, yerr=1,fmt='s', capsize=3, capthick=3, color='dimgray', label='Simulation')
for i in range(len(dim)-1, -1, -1):
    ax.errorbar(x=-10, y=-10, fmt='s', color=colors[i], label='Dim. = {:4d}'.format(dim[i]))
    
ax.set_xticks(np.arange(len(drprate)));
ax.set_xticklabels(['{:.2f}'.format(p) for p in drprate]);
ax.set_xlabel('Dropout Rates')
ax.set_ylabel('Sum of Squared Error (SSE)');
ax.legend(loc='best');

ax.set_xlim(xmin=-0.5)
ax.set_ylim(ymin=-1.5)

plt.tight_layout()
plt.savefig('lr-synthetic.png', format='png', dpi=300)